In [1]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

In [2]:
import sys
import os

# Get the absolute path to the project directory
project_dir = os.path.abspath("..")

# Append the project directory to sys.path
if project_dir not in sys.path:
    sys.path.append(project_dir)

import yfinance as yf
import pandas as pd
import datetime as dt
import matplotlib.pyplot as plt
import msgpack
import warnings
from dataclasses import asdict, is_dataclass, dataclass
import os
import msgpack
import json
import pickle
from pandas.api.types import is_datetime64_any_dtype

from src.common.AssetData import AssetData
from src.common.AssetDataService import AssetDataService
from src.common.AssetFileInOut import AssetFileInOut 
from src.databaseService.OutsourceLoader import OutsourceLoader

from src.simulation.SimulatePortfolio import SimulatePortfolio
from src.strategy.StratBuyAndHold import StratBuyAndHold
from src.simulation.ResultAnalyzer import ResultAnalyzer
from src.common.AssetFileInOut import AssetFileInOut


In [11]:
from src.common.YamlTickerInOut import YamlTickerInOut

snp500tickers = YamlTickerInOut("../src/tickerSelection").loadFromFile("snp500.yaml")["snp500tickers"]

snp500tickers[0:5]

['MMM', 'AOS', 'ABT', 'ABBV', 'ACN']

In [4]:
a=pd.date_range(pd.Timestamp(2010,1,1), pd.Timestamp(2010,2,1), freq='B', tz='UTC')
a

DatetimeIndex(['2010-01-01 00:00:00+00:00', '2010-01-04 00:00:00+00:00',
               '2010-01-05 00:00:00+00:00', '2010-01-06 00:00:00+00:00',
               '2010-01-07 00:00:00+00:00', '2010-01-08 00:00:00+00:00',
               '2010-01-11 00:00:00+00:00', '2010-01-12 00:00:00+00:00',
               '2010-01-13 00:00:00+00:00', '2010-01-14 00:00:00+00:00',
               '2010-01-15 00:00:00+00:00', '2010-01-18 00:00:00+00:00',
               '2010-01-19 00:00:00+00:00', '2010-01-20 00:00:00+00:00',
               '2010-01-21 00:00:00+00:00', '2010-01-22 00:00:00+00:00',
               '2010-01-25 00:00:00+00:00', '2010-01-26 00:00:00+00:00',
               '2010-01-27 00:00:00+00:00', '2010-01-28 00:00:00+00:00',
               '2010-01-29 00:00:00+00:00', '2010-02-01 00:00:00+00:00'],
              dtype='datetime64[ns, UTC]', freq='B')

In [6]:
a=pd.Timestamp(2010,1,1)
f"hello {a.tz}"

'hello None'